In [1]:
"""
Fork of https://www.kaggle.com/antmarakis/bi-lstm-conv-layer?scriptVersionId=2789290
Just replaced the data with Preprocessed data
Public LB score 0.9833 => 0.9840

"""

import numpy as np
import pandas as pd
from keras.layers import Dense, Input, LSTM, Bidirectional, Conv1D, CuDNNLSTM
from keras.layers import Dropout, Embedding
from keras.preprocessing import text, sequence
from keras.layers import GlobalMaxPooling1D, GlobalAveragePooling1D, concatenate, SpatialDropout1D
from keras.models import Model
from unidecode import unidecode
import re

train = pd.read_csv('../data/train.csv')
test = pd.read_csv('../data/test.csv')

Using TensorFlow backend.


In [2]:
import xgboost as xgb

In [3]:
special_character_removal=re.compile(r'[^a-z\?\!\#\@\%\* ]',re.IGNORECASE)
def clean_text(x):
    x_ascii = unidecode(x)
    x_clean = special_character_removal.sub('',x_ascii)
    return x_clean

train['clean_text'] = train['comment_text'].apply(lambda x: clean_text(str(x)))
test['clean_text'] = test['comment_text'].apply(lambda x: clean_text(str(x)))

In [4]:
max_features=100000
maxlen=150
embed_size=300

# max_features=50000
# maxlen=900
# embed_size=301

train_y = train[['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']].values
train_x = train['clean_text']#.str.lower()

test_x = test['clean_text']#.str.lower()

In [5]:
# Vectorize text + Prepare fasttext Embedding
tokenizer = text.Tokenizer(num_words=max_features, lower=True)
tokenizer.fit_on_texts(list(train_x))

train_x = tokenizer.texts_to_sequences(train_x)
test_x = tokenizer.texts_to_sequences(test_x)

train_x = sequence.pad_sequences(train_x, maxlen=maxlen)
test_x = sequence.pad_sequences(test_x, maxlen=maxlen)

In [6]:
# for i in train_x[0]:
#     print(list(tokenizer.word_index.keys())[list(tokenizer.word_index.values()).index(i)])

In [7]:
EMBEDDING_FILE="../data/fasttext/crawl-300d-2M.vec"
embeddings_index = {}
with open(EMBEDDING_FILE, encoding='utf8') as f:
    for line in f:
        values = line.rstrip().rsplit(' ')
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        embeddings_index[word] = coefs

word_index = tokenizer.word_index
num_words = min(max_features, len(word_index) + 1)
embedding_matrix = np.zeros((num_words, embed_size))

for word, i in word_index.items():
    if i >= max_features:
        continue
    
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

In [20]:
embedding_matrix[0,:]

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0.

In [24]:
# Build Model
inp = Input(shape=(maxlen,))

x = Embedding(max_features, embed_size, weights=[embedding_matrix], trainable=False)(inp)
x = SpatialDropout1D(0.35)(x)

x = Bidirectional(CuDNNLSTM(128, return_sequences=True))(x)
#x = Dropout(0.3)(x) #dropout doesnt improve
x = Conv1D(64, kernel_size=3, padding='valid', kernel_initializer='glorot_uniform')(x)

avg_pool = GlobalAveragePooling1D()(x)
max_pool = GlobalMaxPooling1D()(x)
x = concatenate([avg_pool, max_pool])

out = Dense(6, activation='sigmoid')(x)

model = Model(inp, out)
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

ResourceExhaustedError: OOM when allocating tensor of shape [] and type float
	 [[Node: embedding_1/random_uniform/sub = Const[dtype=DT_FLOAT, value=Tensor<type: float shape: [] values: 0.1>, _device="/job:localhost/replica:0/task:0/device:GPU:0"]()]]

Caused by op 'embedding_1/random_uniform/sub', defined at:
  File "/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/ipykernel/__main__.py", line 3, in <module>
    app.launch_new_instance()
  File "/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/ipykernel/kernelapp.py", line 486, in

In [7]:
# Prediction
batch_size = 512
epochs = 5

model.fit(train_x, train_y, batch_size=batch_size, epochs=epochs, verbose=1)

Epoch 1/5
159571/159571 [==============================] - 422s 3ms/step - loss: 0.0905 - acc: 0.9721
Epoch 2/5
159571/159571 [==============================] - 422s 3ms/step - loss: 0.0479 - acc: 0.9825
Epoch 3/5
159571/159571 [==============================] - 422s 3ms/step - loss: 0.0422 - acc: 0.9841
Epoch 4/5
159571/159571 [==============================] - 422s 3ms/step - loss: 0.0382 - acc: 0.9853
Epoch 5/5
159571/159571 [==============================] - 421s 3ms/step - loss: 0.0346 - acc: 0.9865


In [8]:
predictions = model.predict(test_x, batch_size=batch_size, verbose=1)

153164/153164 [==============================] - 127s 831us/step


In [9]:
# Create submission file
output=pd.DataFrame(data=predictions, index=test["id"])
output.to_csv("./output/bilstm_conv_embedding_matrix.csv",header=["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]
              ,index=True)